# Creating A New MLRun Project
  --------------------------------------------------------------------

creating a full project with multiple functions and workflow and working wit Git.

#### **notebook how-to's**
* Add local or library/remote functions
* Add a workflow
* Save to a remote git
* Run pipeline

<a id='top'></a>
#### **steps**
**[Add functions](#load-functions)**<br>
**[Create and save a workflow](#create-workflow)**<br>
**[Update remote git](#git-remote)**<br>
**[Run a pipeline workflow](#run-pipeline)**<br>

In [1]:
from mlrun import new_project, new_function, mount_v3io

In [2]:
# update the dir and repo to reflect real locations 
# the remote git repo must be initialized in GitHub
project_dir = '/User/new-proj'
remote_git = 'https://github.com/yaronha/mlp2.git'
newproj = new_project('new-project', project_dir, init_git=True)

Set the remote git repo and pull to sync in case it has some content

In [3]:
newproj.create_remote(remote_git)

GitCommandError: Cmd('git') failed due to: exit code(128)
  cmdline: git remote add origin https://github.com/yaronha/mlp2.git
  stderr: 'fatal: remote origin already exists.'

In [4]:
newproj.pull()

<a id='load-functions'></a>
### Load functions from remote URLs
We create two functions:
1. Load a function from a remote notebook url ( converted into a function object)
2. Create a function from file in the context dir (w copy a demo file into the dir) 

In [5]:
newproj.set_function('https://raw.githubusercontent.com/mlrun/functions/master/serving/xgboost/xgb_serving.ipynb', 'serving')

In [6]:
!curl -o {project_dir}/handler.py https://raw.githubusercontent.com/mlrun/mlrun/master/examples/handler.py

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   513  100   513    0     0   8127      0 --:--:-- --:--:-- --:--:--  8142


In [7]:
new_func = new_function(name='tstfunc', command='handler.py', kind='job')
new_func.spec.build.base_image = 'mlrun/mlrun'
newproj.set_function(new_func)
print(newproj.func('tstfunc').to_yaml())

kind: job
metadata:
  name: tstfunc
  project: new-project
spec:
  command: handler.py
  args: []
  image: ''
  volumes: []
  volume_mounts: []
  env: []
  description: ''
  build:
    source: git://github.com/yaronha/mlp2.git#refs/heads/master
    base_image: mlrun/mlrun
    commands: []
    code_origin: https://github.com/yaronha/mlp2.git#bc4adced86588da3354d5d5d2cc5feece83d8310



<a id='create-workflow'></a>
### Create a workflow file and store it in the context dir

In [8]:
%%writefile {project_dir}/workflow.py
from kfp import dsl
from mlrun import mount_v3io

artifacts_path = './'
funcs = {}

def init_functions(functions: dict, params=None, secrets=None):
    pass

@dsl.pipeline(
    name='demo project', description='Shows how to use mlrun project.'
)
def kfpipeline(p1=3):
    # first step build the function container
    builder = funcs['tstfunc'].deploy_step(with_mlrun=False)

    # first step
    s1 = funcs['tstfunc'].as_step(name='step-one', handler='my_func',
         image=builder.outputs['image'],
         params={'p1': p1}, out_path=artifacts_path)


Overwriting /User/new-proj/workflow.py


In [10]:
newproj.set_workflow('main', 'workflow.py')

In [11]:
print(newproj.to_yaml())

name: new-project
functions:
- url: https://raw.githubusercontent.com/mlrun/functions/master/serving/xgboost/xgb_serving.ipynb
  name: serving
- name: tstfunc
  spec:
    kind: job
    metadata:
      name: tstfunc
      project: new-project
    spec:
      command: handler.py
      args: []
      image: ''
      env: []
      description: ''
      build:
        source: ./
        base_image: mlrun/mlrun
        commands: []
        code_origin: https://github.com/yaronha/mlp2.git#bc4adced86588da3354d5d5d2cc5feece83d8310
workflows:
- name: main
  code: workflow.py



<a id='git-remote'></a>
### Register and push the project to a remote Repo

In [12]:
newproj.push('master', 'first push', add=['handler.py', 'workflow.py'])

In [13]:
newproj.source

'git://github.com/yaronha/mlp2.git#refs/heads/master'

<a id='run-pipeline'></a>
### Run the workflow

In [14]:
newproj.run('main', arguments={}, artifact_path='v3io:///users/admin/mlrun/kfp/{{workflow.uid}}/', )

/User/.pythonlibs/jupyter/lib/python3.6/site-packages/kfp/components/_data_passing.py:133: UserWarning: Missing type name was inferred as "Integer" based on the value "3".
  warnings.warn('Missing type name was inferred as "{}" based on the value "{}".'.format(type_name, str(value)))


[mlrun] 2020-02-27 15:24:40,697 Pipeline run id=adfa389b-faef-4e56-8b89-ced0f7c97937, check UI or DB for progress


'adfa389b-faef-4e56-8b89-ced0f7c97937'

**[back to top](#top)**